In [1]:
#%load_ext cython
%load_ext line_profiler

In [2]:
import numpy as np
import tree_build_routines_c as tbrc
import list_build_routines_c as lbrc
import tree_compute_routines_c as tcrc
import tree_build_routines as tbr
import list_build_routines as lbr
import tree_compute_routines as tcr
import pstats, cProfile
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
npts = 50

x = np.linspace(1.,2.,npts+1) 
zpos, xpos = np.meshgrid(x, x)
gvals = 1e-2*np.ones((npts+1)**2) 
pval = 10
mx = 5.
ep = 1./npts
nvorts = xpos.size
ccnt, rcnt = 2, 2

mytree = tbr.make_tree(xpos.flatten(), zpos.flatten(), gvals, pval, mx, ep, nvorts, ccnt, rcnt)
tbr.build_tree(mytree)
lbr.build_tree_lists(mytree)
Kvals = tcr.multipole_comp(mytree)
# print Kvals

%timeit -t tbr.build_tree(mytree)
%timeit -t lbr.build_tree_lists(mytree)
%timeit -t tcr.multipole_comp(mytree)

# Kvalss = tcrc.near_neighbor_comp(xpos.flatten(), zpos.flatten(), np.empty(0), np.empty(0), gvals, np.empty(0), mx, ep, nvorts, 0)
# print np.linalg.norm(Kvals-Kvalss,np.inf)/np.linalg.norm(Kvalss,np.inf)
# print (Kvalss-Kvals)/np.linalg.norm(Kvalss,np.inf)
# print np.linalg.norm(Kvals-Kvalss, np.inf)
# diff = np.max(np.abs(Kvals-Kvalss) ,axis=1)
# plt.figure(1)
# plt.plot(np.arange(nvorts),diff)


10 loops, best of 3: 147 ms per loop
10 loops, best of 3: 81.2 ms per loop
10 loops, best of 3: 174 ms per loop


In [4]:
mytreec = tbrc.make_tree(xpos.flatten(), zpos.flatten(), gvals, pval, mx, ep, nvorts, ccnt, rcnt)
tbrc.build_tree(mytreec)
lbrc.build_tree_lists(mytreec)
# Kvalsc = tcrc.multipole_comp(mytreec)

%timeit -t tbrc.build_tree(mytreec)
%timeit -t lbrc.build_tree_lists(mytreec)
%timeit -t tcrc.multipole_comp(mytreec)

# %timeit -t tcr.near_neighbor_comp(xpos.flatten(), zpos.flatten(), np.empty(0), np.empty(0), gvals, np.empty(0), ep, mx, nvorts, 0)
# %timeit -t tcrc.near_neighbor_comp(xpos.flatten(), zpos.flatten(), np.empty(0), np.empty(0), gvals, np.empty(0), ep, mx, nvorts, 0)

# Kvalsfst = tcrc.near_neighbor_comp(xpos.flatten(), zpos.flatten(), np.empty(0), np.empty(0), gvals, np.empty(0), mx, ep, nvorts, 0)
# print Kvalsfst

# %lprun -f lbr.build_node_lists lbr.build_tree_lists(mytree)
cProfile.runctx("tcrc.multipole_comp(mytreec)", globals(), locals(), "Profile.prof")
s = pstats.Stats("Profile.prof")
s.strip_dirs().sort_stats("time").print_stats()

1 loop, best of 3: 231 ms per loop
10 loops, best of 3: 136 ms per loop
10 loops, best of 3: 150 ms per loop
Fri Nov 30 13:21:14 2018    Profile.prof

         4779 function calls in 0.181 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.158    0.158    0.181    0.181 {tree_compute_routines_c.multipole_comp}
     2388    0.012    0.000    0.012    0.000 {numpy.core.multiarray.array}
     2388    0.011    0.000    0.022    0.000 numeric.py:433(asarray)
        1    0.000    0.000    0.181    0.181 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




<pstats.Stats instance at 0x7f73230ea050>